In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mydata/MOBILE.xlsx
/kaggle/input/mydata/pattern_replacements.json
/kaggle/input/mydata/text2.txt
/kaggle/input/mydata/query-hive-415.xlsx
/kaggle/input/mydata/FIXE.xlsx
/kaggle/input/mydata/cleaned_text_combined_mobile.pkl
/kaggle/input/mydata/Mooli-Regular.ttf
/kaggle/input/mydata/DATASET_HS_2023.xlsx
/kaggle/input/mydata/Nuages_de_mots.xlsx
/kaggle/input/mydata/frenchStopwords2.txt
/kaggle/input/mydata/text.txt
/kaggle/input/mydata/cleaned_text_combined.pkl
/kaggle/input/mydata/text3.txt


In [2]:
!pip install spacy-lefff

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 9.3 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for msgpack: filename=msgpack-0.5.6-cp310-cp310-linux_x86_64.whl size=13845 sha256=355382c399a502f0d45735f249e02c025d8e5c6bf1596019261d4ac14fcb1b8e
  Stored in directory: /root/.cache/pip/wheels/9d/27/cf/62a0a8f89d65ce9e28e3806cb3479f110d32a1c9cb0cdd6daf
Successfully built msgpack
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.5
    Uninstalling msgpack-1.0.5:
      Successfully uninstalled msgpack-1.0.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2023.9.0 requires msgpack>=1.0.0, but you have msgpack 0.5.6 which is 

In [3]:
!python -m spacy download fr_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 42.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import CoherenceModel
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
nltk.download('wordnet')
nltk.download('omw-1.4')
import matplotlib.pyplot as plt
import os
import re
import spacy
from PIL import Image
import nltk
from concurrent.futures import ThreadPoolExecutor
import json

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [5]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk

True

In [6]:
from spacy.language import Language
from spacy_lefff import LefffLemmatizer

@Language.factory('french_lemmatizer')
def create_french_lemmatizer(nlp, name):
    return LefffLemmatizer()
nlp = spacy.load('fr_core_news_sm')
nlp.add_pipe('french_lemmatizer', name='lefff')

In [ ]:
# Load the French stopwords
with open('/kaggle/input/mydata/frenchStopwords2.txt', 'r', encoding='utf-8') as file:
    frenchStopwords = file.read().split('\n')

# Combine French stopwords with NLTK stopwords
all_stopwords = frenchStopwords + stopwords.words('french')


# Define a folder for uploading files
#UPLOAD_FOLDER = 'uploads'
#if not os.path.exists(UPLOAD_FOLDER):
#    os.makedirs(UPLOAD_FOLDER)

# Read the text file
#with open('/kaggle/input/mydata/text2.txt', 'r', encoding='utf-8') as file:
#    lines = file.readlines()

# Load data from Excel file
excel_file_path = '/kaggle/input/mydata/MOBILE.xlsx'  # Specify the path to your Excel file
df = pd.read_excel(excel_file_path)

# Define a list of patterns to search for and their corresponding replacement

# Chargement des remplacements depuis le fichier JSON
with open('/kaggle/input/mydata/pattern_replacements.json', 'r', encoding='utf-8') as file:
    pattern_replacements = json.load(file)

def merge_similar_phrases(text):
    for pattern, replacement in pattern_replacements:
        text = re.sub(pattern, replacement, text)
    return text

# Assuming your descriptions are in a column named 'description' in the Excel file
descriptions = df['Description'].tolist()

# Function to clean and process text (similar to your previous code)
def clean_text(text):
    # Apply the pattern replacements first
    text_without_letter_apostrophe = re.sub(r"\b(?:[a-zA-Z])\'\b", '', text)
    text_lower = text_without_letter_apostrophe.lower()
    cleantext = merge_similar_phrases(text_lower)
    # Tokenization (dividing the text into words)
    words = nltk.word_tokenize(cleantext)  # Convert to lowercase
    # Remove punctuation, stopwords, and non-alphabetic words
    cleaned_words = [word for word in words if word not in all_stopwords and word.isalpha()]
    # Lemmatization
    doc = nlp(" ".join(cleaned_words))
    lemmatized_words = [token.lemma_ for token in doc]
    # Return a list of lemmatized words instead of a single string
    return lemmatized_words



#documents = text

# Divisez les lignes en lots (par exemple, 10 lignes par lot)
batch_size = 10000
description_batches = [descriptions[i:i + batch_size] for i in range(0, len(descriptions), batch_size)]

# Fonction pour traiter une ligne
def process_line(lines):
    cleaned_text_list = []
    for line in lines:
         if isinstance(line, str):
                # Nettoyez et traitez la ligne comme un document LDA
                cleaned_text = clean_text(line)
                cleaned_text_list.append(cleaned_text)
    return cleaned_text_list


# Créez un ThreadPoolExecutor avec un certain nombre de threads
num_threads = 3  # Vous pouvez ajuster le nombre de threads en fonction de votre matériel
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Traitez chaque lot de lignes en parallèle
    cleaned_text_list = list(executor.map(process_line, description_batches))

# Combinez les résultats en une seule liste de texte nettoyé pour LDA
cleaned_text_combined = [text for batch in cleaned_text_list for text in batch]


In [3]:
import pickle
# Spécifiez le chemin du fichier dans lequel vous souhaitez enregistrer la liste
output_file_path = '/kaggle/working/cleaned_text_combined_mobile.pkl'

# Enregistrez la liste dans le fichier
with open(output_file_path, 'wb') as file:
    pickle.dump(cleaned_text_combined, file)

print(f'La liste a été enregistrée dans {output_file_path}')


NameError: name 'cleaned_text_combined' is not defined

In [12]:
import pickle
# Spécifiez le chemin du fichier contenant la liste sauvegardée
input_file_path = '/kaggle/input/mydata/cleaned_text_combined_mobile.pkl'

# Chargez la liste à partir du fichier
with open(input_file_path, 'rb') as file:
    cleaned_text_combined = pickle.load(file)


In [6]:

# Convertissez votre texte nettoyé en une liste de listes de mots (tokens)
#cleanedTextList = [cleanedText.split()]

# Créez le dictionnaire à partir de la liste de listes de mots
dictionary = gensim.corpora.Dictionary(cleaned_text_combined)

#dictionary.filter_extremes(no_below=15, no_above=0.6, keep_n=1000)
bow_corpus = [dictionary.doc2bow(doc) for doc in cleaned_text_combined]
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=12, id2word=dictionary, passes=25)
topics = []
for idx, topic in lda_model.print_topics(-1) :
    print("Topic: {} -> Words: {}".format(idx, topic))
    topics.append(topic)

coherence_model_lda = CoherenceModel(model=lda_model, texts=cleaned_text_combined, dictionary=dictionary)
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)


Topic: 0 -> Words: 0.214*"contact" + 0.117*"service" + 0.082*"appel" + 0.075*"activer" + 0.054*"commun" + 0.048*"souscrir" + 0.045*"mixel" + 0.043*"numéro" + 0.040*"desactiver" + 0.023*"quartier"
Topic: 1 -> Words: 0.124*"connecter" + 0.103*"nombre" + 0.078*"autre" + 0.076*"changer" + 0.057*"profil" + 0.045*"ba" + 0.042*"application" + 0.033*"utiliser" + 0.031*"cours" + 0.031*"voir"
Topic: 2 -> Words: 0.122*"promo" + 0.069*"favori" + 0.066*"perte" + 0.065*"credit" + 0.063*"transfert" + 0.035*"sms" + 0.027*"argent" + 0.027*"désactiver" + 0.026*"crédit" + 0.023*"souscription"
Topic: 3 -> Words: 0.159*"demande" + 0.064*"illimi" + 0.056*"hors" + 0.037*"num" + 0.032*"receiver" + 0.031*"etat" + 0.031*"remboursement" + 0.029*"sender" + 0.027*"info" + 0.021*"mpec"
Topic: 4 -> Words: 0.217*"pas" + 0.093*"internet" + 0.085*"forfaire" + 0.070*"constater" + 0.070*"terme" + 0.059*"épuiser" + 0.043*"epuise" + 0.042*"pass" + 0.027*"vite" + 0.023*"education"
Topic: 5 -> Words: 0.143*"transaction" + 0.

In [9]:
from gensim import corpora
from gensim.models import Phrases
from gensim.models.phrases import Phraser

In [16]:
# Créez des bigrammes
bigram = Phrases(cleaned_text_combined, min_count=50, threshold=100)

# Créez des trigrammes en utilisant les bigrammes
#trigram = Phrases(bigram[cleaned_text_combined], threshold=100)

# Appliquez les bigrammes et trigrammes à vos documents
bigram_mod = Phraser(bigram)
#trigram_mod = Phraser(trigram)

# Appliquez les bigrammes et trigrammes à chaque document
bigram_documents = [bigram_mod[doc] for doc in cleaned_text_combined]
#trigram_documents = [trigram_mod[bigram_mod[doc]] for doc in cleaned_text_combined]


In [22]:
# Créez un nouveau dictionnaire à partir des trigrammes
dictionary_with_bigrams = corpora.Dictionary(bigram_documents)

# Créez un nouveau corpus en utilisant le dictionnaire avec les trigrammes
bow_corpus_with_bigrams = [dictionary_with_bigrams.doc2bow(doc) for doc in bigram_documents]

# Entraînez votre modèle LDA avec le corpus et le dictionnaire mis à jour
lda_model_with_bigrams = gensim.models.LdaMulticore(bow_corpus_with_bigrams, num_topics=12, id2word=dictionary_with_bigrams, passes=25)

topics = []
for idx, topic in lda_model_with_bigrams.print_topics(-1) :
    print("Topic: {} -> Words: {}".format(idx, topic))
    topics.append(topic)

coherence_model_lda_with_bigrams = CoherenceModel(model=lda_model_with_bigrams, texts=bigram_documents, dictionary=dictionary_with_bigrams)
coherence_lda_with_bigrams = coherence_model_lda_with_bigrams.get_coherence()
print('Coherence Score: ', coherence_lda_with_bigrams)


Topic: 0 -> Words: 0.113*"pas" + 0.111*"constater" + 0.078*"forfaire" + 0.071*"terme" + 0.059*"épuiser" + 0.044*"cours" + 0.042*"epuise" + 0.034*"barre" + 0.027*"vite" + 0.021*"so"
Topic: 1 -> Words: 0.240*"pas" + 0.100*"acheter" + 0.089*"achat" + 0.077*"sos" + 0.069*"credit" + 0.050*"souscrir" + 0.034*"internet" + 0.033*"mois" + 0.028*"crédit" + 0.027*"type"
Topic: 2 -> Words: 0.258*"transaction" + 0.134*"bénéficiaire" + 0.089*"erreur" + 0.077*"om" + 0.065*"faire" + 0.046*"annuler" + 0.046*"transfert" + 0.039*"effectuer" + 0.020*"changer" + 0.018*"montant"
Topic: 3 -> Words: 0.054*"pass" + 0.052*"illimi" + 0.047*"box" + 0.039*"pas" + 0.033*"connexion" + 0.025*"cnx" + 0.022*"signal" + 0.021*"recharger" + 0.021*"modem" + 0.020*"souscrir"
Topic: 4 -> Words: 0.094*"souci" + 0.072*"recevoir" + 0.056*"bonus" + 0.055*"reseau" + 0.046*"réseau" + 0.036*"appel" + 0.023*"ref" + 0.021*"remboursement" + 0.019*"rechargemer" + 0.017*"connaitre"
Topic: 5 -> Words: 0.155*"service" + 0.077*"concerner" 

In [19]:
# Enregistrez le modèle LDA dans un fichier
model_path = "/kaggle/working/lda_model_with_bigrams"
lda_model_with_bigrams.save(model_path)

# Vous pouvez également sauvegarder le dictionnaire si nécessaire
#dictionary_path = "/kaggle/working/dictionarymobile439"
#dictionary.save(dictionary_path)

# Pour charger le modèle plus tard, vous pouvez utiliser la commande suivante
#loaded_lda_model = gensim.models.LdaMulticore.load(model_path)

# Pour charger le dictionnaire
#loaded_dictionary = gensim.corpora.Dictionary.load(dictionary_path)


In [ ]:
import random
from sklearn.model_selection import ParameterSampler

# Créez le dictionnaire à partir de la liste de listes de mots
dictionary = gensim.corpora.Dictionary(cleaned_text_combined)

dictionary.filter_extremes(no_below=15, no_above=0.2, keep_n=100)
bow_corpus = [dictionary.doc2bow(doc) for doc in cleaned_text_combined]
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=12, id2word=dictionary, passes=100)
topics = []

# Définir l'espace des hyperparamètres que vous souhaitez optimiser
param_space = {
    'num_topics': [5, 10, 15, 20],
    'passes': [50, 100, 150],
    'alpha': ['auto', 0.01, 0.1, 1.0],
    'eta': ['auto', 0.01, 0.1, 1.0],
}

# Nombre d'itérations de recherche aléatoire
n_iterations = 10

# Liste pour stocker les résultats
results = []

# Effectuer la recherche aléatoire
for _ in range(n_iterations):
    # Échantillon aléatoire des hyperparamètres
    params = random.choice(list(ParameterSampler(param_space, n_iter=1)))

    # Utiliser les hyperparamètres pour ajuster un modèle LDA
    lda_model = gensim.models.LdaModel(bow_corpus, id2word=dictionary, **params)

    # Évaluer le modèle (par exemple, avec la cohérence des sujets)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=cleaned_text_combined, dictionary=dictionary)
    coherence_lda = coherence_model_lda.get_coherence()

    # Enregistrer les résultats
    results.append({'params': params, 'coherence': coherence_lda})

# Triez les résultats par cohérence décroissante
results.sort(key=lambda x: x['coherence'], reverse=True)

# Affichez les résultats triés
for result in results:
    print("Hyperparameters:", result['params'])
    print("Coherence Score:", result['coherence'])


In [1]:
# Load the French stopwords
with open('/kaggle/input/mydata/frenchStopwords2.txt', 'r', encoding='utf-8') as file:
    frenchStopwords = file.read().split('\n')

# Combine French stopwords with NLTK stopwords
all_stopwords = frenchStopwords + stopwords.words('french')


# Define a folder for uploading files
#UPLOAD_FOLDER = 'uploads'
#if not os.path.exists(UPLOAD_FOLDER):
#    os.makedirs(UPLOAD_FOLDER)

# Read the text file
#with open('/kaggle/input/mydata/text2.txt', 'r', encoding='utf-8') as file:
#    lines = file.readlines()

# Load data from Excel file
excel_file_path = '/kaggle/input/mydata/MOBILE.xlsx'  # Specify the path to your Excel file
df = pd.read_excel(excel_file_path)

# Define a list of patterns to search for and their corresponding replacement

pattern_replacements = [

    (r'\bpositionn\b', 'positionner'),

    (r'\brechargemer\b', 'rechargement'),

    (r'\bbloqu\b', 'bloque'),

    (r'\brechar\b', 'retrait'),

    (r'\bretraire\b', 'retrait'),

    (r'\bstart\b', 'star'),

    (r'\bsouscrir\b', 'souscription'),

    (r'\bsauvegard\b', 'sauvegarde'),

    (r'\brepertoir\b', 'repertoire'),

    (r'\brecu\b', 'reçu'),

    (r'\bperdre\b', 'perdu'),  # Replace "perdre" with "perdu"
    (r'\bcredit\b', 'crédit'),  # Replace "perdre" with "perdu"

    (r'\bcas\b', 'case'),

    (r'\bpas\b', 'pass'),

    (r'\bsouscrir\b', 'souscription'),

    (r'\billimi\b', 'illimix'),

    (r'\bperte\b', 'perdu'),   # Replace "perte" with "perdu"

    (r'\bperdre\u00A0', 'perdu'),  # Replace "perdre" with "perdu"

    (r'\bperte\u00A0', 'perdu'),   # Replace "perte" with "perdu"
    (r'\bbénéficiaire faux\b', 'faux bénéficiaire'),
    (r'\bfaire sos\b', 'souscrire sos'),
    (r'\bsouscrir sos\b', 'souscrire sos'),
    (r'\bcnx\b', 'connexion'),
    (r'\bpasser\b', 'pass'),
    (r'\bchanger mot\b', 'changer mdp'),
    # Add more patterns as needed
]

def merge_similar_phrases(text):
    for pattern, replacement in pattern_replacements:
        text = re.sub(pattern, replacement, text)
    return text

# Assuming your descriptions are in a column named 'description' in the Excel file
descriptions = df['Description'].tolist()

# Function to clean and process text (similar to your previous code)
def clean_text(text):
    # Apply the pattern replacements first
    text = merge_similar_phrases(text)
    
    text_without_letter_apostrophe = re.sub(r"\b(?:[a-zA-Z])\'\b", '', text)
    # Tokenization (dividing the text into words)
    words = nltk.word_tokenize(text_without_letter_apostrophe.lower())  # Convert to lowercase
    # Remove punctuation, stopwords, and non-alphabetic words
    cleaned_words = [word for word in words if word not in all_stopwords and word.isalpha()]
    # Lemmatization
    doc = nlp(" ".join(cleaned_words))
    lemmatized_words = [token.lemma_ for token in doc]
    # Return a list of lemmatized words instead of a single string
    return lemmatized_words



#documents = text

# Divisez les lignes en lots (par exemple, 10 lignes par lot)
batch_size = 10000
description_batches = [descriptions[i:i + batch_size] for i in range(0, len(descriptions), batch_size)]

# Fonction pour traiter une ligne
def process_line(lines):
    cleaned_text_list = []
    for line in lines:
         if isinstance(line, str):
                # Nettoyez et traitez la ligne comme un document LDA
                cleaned_text = clean_text(line)
                cleaned_text_list.append(cleaned_text)
    return cleaned_text_list


# Créez un ThreadPoolExecutor avec un certain nombre de threads
num_threads = 3  # Vous pouvez ajuster le nombre de threads en fonction de votre matériel
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Traitez chaque lot de lignes en parallèle
    cleaned_text_list = list(executor.map(process_line, description_batches))

# Combinez les résultats en une seule liste de texte nettoyé pour LDA
cleaned_text_combined = [text for batch in cleaned_text_list for text in batch]


In [15]:
# Enregistrez le modèle LDA dans un fichier
model_path = "/kaggle/working/model_mobile10topics_435"
lda_model.save(model_path)

# Vous pouvez également sauvegarder le dictionnaire si nécessaire
dictionary_path = "/kaggle/working/dictionarymobile"
dictionary.save(dictionary_path)

# Pour charger le modèle plus tard, vous pouvez utiliser la commande suivante
#loaded_lda_model = gensim.models.LdaMulticore.load(model_path)

# Pour charger le dictionnaire
#loaded_dictionary = gensim.corpora.Dictionary.load(dictionary_path)


In [3]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis


In [11]:
# Créez la visualisation
lda_display = gensimvis.prepare(lda_model, bow_corpus, dictionary)

# Affichez la visualisation dans votre notebook
pyLDAvis.display(lda_display)


TypeError: DataFrame.drop() takes from 1 to 2 positional arguments but 3 were given

In [ ]:
# Définissez la taille de groupe (100 lignes par document)
group_size = 100
documents = []

# Initialiser un document en cours
current_document = ""

# Parcourir les lignes de texte
for line in text:
    current_document += line + "\n"  # Ajouter la ligne actuelle au document en cours

    # Si nous avons atteint le nombre de lignes souhaité, ajouter le document en cours à la liste des documents
    if len(current_document.split('\n')) >= group_size:
        documents.append(current_document)
        current_document = ""  # Réinitialiser le document en cours

# Ajouter le dernier document si nécessaire (s'il n'est pas vide)
if current_document:
    documents.append(current_document)

# Function to clean and process text (similar to your previous code)
def clean_text(text):
    text_without_letter_apostrophe = re.sub(r"\b(?:[a-zA-Z])\'\b", '', text)
    # Tokenization (dividing the text into words)
    words = nltk.word_tokenize(text_without_letter_apostrophe.lower())  # Convert to lowercase
    # Remove punctuation, stopwords, and non-alphabetic words
    cleaned_words = [word for word in words if word not in all_stopwords and word.isalpha()]
    # Lemmatization
    doc = nlp(" ".join(cleaned_words))
    lemmatized_words = [token.lemma_ for token in doc]
    # Return a list of lemmatized words instead of a single string
    return lemmatized_words


# Traitement du texte ici
cleanedTextList = [clean_text(doc) for doc in documents]
# Convertissez votre texte nettoyé en une liste de listes de mots (tokens)
#cleanedTextList = [cleanedText.split()]

# Créez le dictionnaire à partir de la liste de listes de mots
dictionary = gensim.corpora.Dictionary(cleanedTextList)
dictionary.filter_extremes(no_below=15, no_above=0.95, keep_n=1000)
print(dictionary)
bow_corpus = [dictionary.doc2bow(doc) for doc in cleanedTextList]
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=100)
topics = []
for idx, topic in lda_model.print_topics(-1) :
    print("Topic: {} -> Words: {}".format(idx, topic))
    topics.append(topic)




In [ ]:
# Import the necessary libraries
#from flask import Flask, request, jsonify, abort
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import os
import re
import spacy
import numpy as np
from PIL import Image
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

nlp = spacy.load("fr_core_news_sm")
nlp.max_length = 13000000  # Augmentez cette valeur en fonction de la longueur de votre texte

#app = Flask(__name__)

# Load the French stopwords
with open('/kaggle/input/mydata/frenchStopwords2.txt', 'r', encoding='utf-8') as file:
    frenchStopwords = file.read().split('\n')

# Combine French stopwords with NLTK stopwords
all_stopwords = frenchStopwords + stopwords.words('french')


# Define a folder for uploading files
#UPLOAD_FOLDER = 'uploads'
#if not os.path.exists(UPLOAD_FOLDER):
#    os.makedirs(UPLOAD_FOLDER)

# Read the text file
with open('/kaggle/input/mydata/text3.txt', 'r', encoding='utf-8') as file:
    text = file.read().split('\n')

# Définissez la taille de groupe (100 lignes par document)
group_size = 100
documents = []

# Initialiser un document en cours
current_document = ""

# Parcourir les lignes de texte
for line in text:
    current_document += line + "\n"  # Ajouter la ligne actuelle au document en cours

    # Si nous avons atteint le nombre de lignes souhaité, ajouter le document en cours à la liste des documents
    if len(current_document.split('\n')) >= group_size:
        documents.append(current_document)
        current_document = ""  # Réinitialiser le document en cours

# Ajouter le dernier document si nécessaire (s'il n'est pas vide)
if current_document:
    documents.append(current_document)

# Function to clean and process text (similar to your previous code)
def clean_text(text):
    text_without_letter_apostrophe = re.sub(r"\b(?:[a-zA-Z])\'\b", '', text)
    # Tokenization (dividing the text into words)
    words = nltk.word_tokenize(text_without_letter_apostrophe.lower())  # Convert to lowercase
    # Remove punctuation, stopwords, and non-alphabetic words
    cleaned_words = [word for word in words if word not in all_stopwords and word.isalpha()]
    # Lemmatization
    doc = nlp(" ".join(cleaned_words))
    lemmatized_words = [token.lemma_ for token in doc]
    # Return a list of lemmatized words instead of a single string
    return lemmatized_words


# Traitement du texte ici
cleanedTextList = [clean_text(doc) for doc in documents]
# Convertissez votre texte nettoyé en une liste de listes de mots (tokens)
#cleanedTextList = [cleanedText.split()]

# Créez le dictionnaire à partir de la liste de listes de mots
dictionary = gensim.corpora.Dictionary(cleanedTextList)
dictionary.filter_extremes(no_below=15, no_above=0.99, keep_n=1000)
print(dictionary)
bow_corpus = [dictionary.doc2bow(doc) for doc in cleanedTextList]
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=100)
topics = []
for idx, topic in lda_model.print_topics(-1) :
    print("Topic: {} -> Words: {}".format(idx, topic))
    topics.append(topic)


